# Neural networks: Classification

Reference:
[Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff)

### Objective:
This lab aims to introduce Keras and TensorFlow - a deep learning API for Python, built on top of TensorFlow,
Keras is built on top of TensorFlow, which provides a convenient way to define and train any kind of deep learning model.

This lab is the initial step to start learning deep learning with python. Try to run the code and see the steps to implement deep learning to solve a real-world problem. You may find some new concepts, do not worry about it now.

## Classifying movie reviews: A binary classification example

Two-class classification, or binary classification, is one of the most common kinds of machine learning problems. In this example, you’ll learn to classify movie reviews as positive or negative, based on the text content of the reviews.

### The IMDB dataset

You’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the Internet Movie Database. They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.

The IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews (sequences of words) have been turned into sequences of integers, where each integer stands for a specific word in a dictionary. This enables us to focus on model building, training, and evaluation.

**Loading the IMDB dataset**

The following code will load the dataset (when you run it the first time, about 80 MB of data will be downloaded to your machine).

In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

From above code :

The argument num_words=10000 means you’ll only keep the top 10,000 most frequently occurring words in the training data. Rare words will be discarded. This allows us to work with vector data of manageable size. If we didn’t set this limit, we’d be working with 88,585 unique words in the training data, which is unnecessarily large. Many of these words only occur in a single sample, and thus can’t be meaningfully used for classification.

The variables train_data and test_data are lists of reviews; each review is a list of word indices (encoding a sequence of words). train_labels and test_labels are lists of 0s and 1s, where 0 stands for negative and 1 stands for positive:

In [ ]:
train_data[0]

In [ ]:
train_labels[0]

Because we’re restricting ourselves to the top 10,000 most frequent words, no word index will exceed 10,000:

In [ ]:
max([max(sequence) for sequence in train_data])

**Decoding reviews back to text**

Here’s how you can quickly decode one of these reviews back to English words.

In [ ]:
word_index = imdb.get_word_index()                                  #❶
reverse_word_index = dict(                                          #❷
    [(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join(                                          #❸
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

❶ word_index is a dictionary mapping words to an integer index.

❷ Reverses it, mapping integer indices to words

❸ Decodes the review. Note that the indices are offset by 3 because 0, 1, and 2 are reserved indices for “padding,” “start of sequence,” and “unknown.”

### Preparing the data

You can’t directly feed lists of integers into a neural network. They all have different lengths, but a neural network expects to process contiguous batches of data.

You have to turn your lists into tensors. One of the way to do that is:

***Multi-hot encode*** your lists to turn them into vectors of 0s and 1s. This would mean, for instance, turning the sequence [8, 5] into a 10,000-dimensional vector that would be all 0s except for indices 8 and 5, which would be 1s. Then you could use a Dense layer, capable of handling floating-point vector data, as the first layer in your model.

**Encoding the integer sequences via multi-hot encoding**

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))   #❶
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.                        #❷
    return results
x_train = vectorize_sequences(train_data)             #❸
x_test = vectorize_sequences(test_data)               #❹

❶ Creates an all-zero matrix of shape (len(sequences), dimension)

❷ Sets specific indices of results[i] to 1s

❸ Vectorized training data

❹ Vectorized test data

Here’s what the samples look like now:

In [ ]:
x_train[0]

You should also vectorize your labels, which is straightforward:

In [ ]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

     Now the data is ready to be fed into a neural network.

### Building your model

The input data is vectors, and the labels are scalars (1s and 0s): this is one of the simplest problem setups you’ll ever encounter. A type of model that performs well on such a problem is a plain stack of densely connected (Dense) layers with relu activations.

There are two key architecture decisions to be made about such a stack of Dense layers:

1. How many layers to use

2. How many units to choose for each layer

There are formal principles to guide you in making these choices. However, this is related to understanding deep learning in details.
For the time being, you’ll implement the following architecture choices:


* Two intermediate layers with 16 units each

* A third layer that will output the scalar prediction regarding the sentiment of the current review

**Model definition**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

The first argument being passed to each Dense layer is the number of units in the layer: the dimensionality of representation space of the layer.

Each such Dense layer with a relu activation implements the following chain of tensor operations:

output = relu(dot(input, W) + b)

Having 16 units means the weight matrix W will have shape (input_dimension, 16): the dot product with W will project the input data onto a 16-dimensional representation space (and then you’ll add the bias vector b and apply the relu operation).

**Compiling the model**

you need to choose a loss function and an optimizer. Because you’re facing a binary classification problem and the output of your model is a probability (you end your model with a single-unit layer with a sigmoid activation), it’s best to use the binary_crossentropy loss. It isn’t the only viable choice: for instance, you could use mean_squared_error. But crossentropy is usually the best choice when you’re dealing with models that output probabilities.


As for the choice of the optimizer, we’ll go with rmsprop, which is a usually a good default choice for virtually any problem.

Here’s the step where we configure the model with the rmsprop optimizer and the binary_crossentropy loss function. Note that we’ll also monitor accuracy during training.

In [ ]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

### Validating your approach

a deep learning model should never be evaluated on its training data—it’s standard practice to use a validation set to monitor the accuracy of the model during training. Here, we’ll create a validation set by setting apart 10,000 samples from the original training data.

**Setting aside a validation set**

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

**Training your model**

We will now train the model for 20 epochs (20 iterations over all samples in the training data) in mini-batches of 512 samples. At the same time, we will monitor loss and accuracy on the 10,000 samples that we set apart. We do so by passing the validation data as the validation_data argument.

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

On CPU, this will take less than 2 seconds per epoch—training is over in 20 seconds. At the end of every epoch, there is a slight pause as the model computes its loss and accuracy on the 10,000 samples of the validation data.

Note that the call to model.fit() returns a History object, as you saw in chapter 3. This object has a member history, which is a dictionary containing data about everything that happened during training. Let’s look at it:

In [ ]:
history_dict = history.history
history_dict.keys()

The dictionary contains four entries: one per metric that was being monitored during training and during validation.

    Now, let’s use Matplotlib to plot the training and validation loss side by side  as well as the training and validation accuracy .

 ***Note that your own results may vary slightly due to a different random initialization of your model.***

**Plotting the training and validation loss**

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")        #❶
plt.plot(epochs, val_loss_values, "b", label="Validation loss")   #❷
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

❶ "bo" is for "blue dot."

❷ "b" is for "solid blue line."

**Plotting the training and validation accuracy**

In [ ]:
plt.clf()                           #❶
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

❶ Clears the figure

As you can see, the training loss decreases with every epoch, and the training accuracy increases with every epoch. That’s what you would expect when running optimization—the quantity you’re trying to minimize should be less with every iteration. But that isn’t the case for the validation loss and accuracy: they seem to peak at the fourth epoch. This is an example of what we warned against earlier: a model that performs better on the training data isn’t necessarily a model that will do better on data it has never seen before. In precise terms, what you’re seeing is overfitting: after the fourth epoch, you’re overoptimizing on the training data, and you end up learning representations that are specific to the training data and don’t generalize to data outside of the training set.

In this case, to prevent overfitting, you could stop training after four epochs.

    Let’s train a new model from scratch for four epochs and then evaluate it on the test data.

**Retraining a model from scratch**

In [ ]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

The final results are as follows:

In [ ]:
results

The first number, 0.29, is the test loss, and the second number, 0.88, is the test accuracy.

This fairly naive approach achieves an accuracy of 88%.

### Using a trained model to generate predictions on new data

After having trained a model, you’ll want to use it in a practical setting. You can generate the likelihood of reviews being positive by using the predict method

In [ ]:
model.predict(x_test)